In [2]:
#Read de original df for BASE
import pandas as pd
dir=r"D:\PROYECTOS 2023\RENE\Villagomez"
FRastreo="21/08/2023"

Dbase=dir+"\\6.Reportes procesamiento\\procesamiento_Base.csv"
Dbase=pd.read_csv(Dbase,sep=',',skiprows=1,encoding='utf-16',thousands=",")
Dbase.drop(columns=Dbase.columns[0],axis=1,inplace=True)


In [3]:
#Read the average file
Daverage=dir+"\\6.Reportes procesamiento\\average.csv"
Daverage=pd.read_csv(Daverage,sep=',',skiprows=1,encoding='utf-16',thousands=",")


In [4]:
#Read de original df for ROVER
Drover=dir+"\\6.Reportes procesamiento\\procesamiento.csv"
Drover=pd.read_csv(Drover,sep=',',skiprows=1,encoding='utf-16',thousands=",")
Drover.drop(columns=Drover.columns[0],axis=1,inplace=True)


In [5]:
#Dataframe XYZ in current epoch
Dxyz0=Daverage[["Point Id","WGS84 Cartesian X [m]","WGS84 Cartesian Y [m]","WGS84 Cartesian Z [m]"]]
Dxyz1=Drover[["Point Id","WGS84 Cartesian X [m]","WGS84 Cartesian Y [m]","WGS84 Cartesian Z [m]"]]
Dxyz=Dxyz0.append(Dxyz1)
Dxyz["F_Rastreo"]=FRastreo;Dxyz["F_Ref"]="01/01/2018"
Dxyz.rename(columns={'WGS84 Cartesian X [m]':'X','WGS84 Cartesian Y [m]':'Y','WGS84 Cartesian Z [m]':'Z'},inplace=True)
Dxyz=Dxyz.reset_index(drop=True)
Dxyz.to_csv(dir+"\\8.Temp\\XYZ.csv",index=False)

In [9]:
#Get Doy and DeltaT
import gnsscal
import  datetime
from  datetime import timedelta
fecha=FRastreo.split("/")
yy=int(fecha[2]);mm = int(fecha[1]);dd = int(fecha[0])
fecha=datetime.date(yy,mm,dd)
DoY=gnsscal.date2doy(fecha) #DoY
Epoch=yy+(DoY/360) #Current Epoch
dT=Epoch-2018.0 #Delta of Time
dEpoch={"Epoca":[Epoch],"Epoca de referencia":[2018.0],"Delta tiempo":[dT]}
dEpoch=pd.DataFrame(dEpoch) #Table Epoch
dEpoch

,Epoca,Epoca de referencia,Delta tiempo
0,2023.647222,2018.0,5.647222


In [7]:
#Get Dataframe of XYZ 2018.0
Dxyz20180=dir+"\\8.Temp\\XYZ_20180.csv"
Dxyz20180=pd.read_csv(Dxyz20180)
Dxyz20180=Dxyz20180[["ID","X","Y","Z"]];Dxyz20180.rename(columns={'X':'X 2018.0','Y':'Y 2018.0','Z':'Z 2018.0'},inplace=True)


In [8]:
#Calculate Velocity XYZ
D_Geo= pd.concat([Dxyz,Dxyz20180],axis=1)
D_Geo=D_Geo[["ID","X","Y","Z","X 2018.0","Y 2018.0","Z 2018.0"]]
D_Geo["Vx"]=((D_Geo["X"]-D_Geo["X 2018.0"])/dT)
D_Geo["Vy"]=((D_Geo["Y"]-D_Geo["Y 2018.0"])/dT)
D_Geo["Vz"]=((D_Geo["Z"]-D_Geo["Z 2018.0"])/dT)
D_Geo=D_Geo[["ID","X","Y","Z","Vx","Vy","Vz","X 2018.0","Y 2018.0","Z 2018.0"]] #Dataframe Geocentric
D_Geo.to_csv(dir+"\\8.Temp\\D_Geo.csv",index=False)

In [2]:
import os
from heapq import nlargest

def get_largest_files(directory, num_files=20):
    file_sizes = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                file_size = os.path.getsize(file_path)
                file_sizes.append((file_size, file_path))
            except (OSError, PermissionError):
                pass
    
    return nlargest(num_files, file_sizes, key=lambda x: x[0])

if __name__ == "__main__":
    disk = "F:/"
    largest_files = get_largest_files(disk)
    
    print("Top 20 archivos más pesados en el disco C:")
    for size, path in largest_files:
        print(f"{size / (1024 ** 3):.2f} GB - {path}")


Top 20 archivos más pesados en el disco C:
2.75 GB - F:/pagefile.sys
0.27 GB - F:/Program Files (x86)\Microsoft\Edge\Application\133.0.3065.51\msedge.dll
0.27 GB - F:/Program Files (x86)\Microsoft\EdgeCore\133.0.3065.51\msedge.dll
0.27 GB - F:/Program Files (x86)\Microsoft\Edge\Application\133.0.3065.59\msedge.dll
0.27 GB - F:/Program Files (x86)\Microsoft\EdgeCore\133.0.3065.59\msedge.dll
0.27 GB - F:/Program Files (x86)\Microsoft\EdgeCore\132.0.2957.140\msedge.dll
0.27 GB - F:/Program Files (x86)\Microsoft\EdgeWebView\Application\132.0.2957.140\msedge.dll
0.25 GB - F:/swapfile.sys
0.21 GB - F:/ProgramData\Package Cache\{73B10333-EAE4-4D44-BC25-22607252B2AD}v3.2.1.3319\Leica.Setup.Infinity.msi
0.21 GB - F:/Windows\Installer\3e4e7.msi
0.19 GB - F:/Windows\System32\MRT.exe
0.11 GB - F:/Windows\servicing\LCU\Package_for_RollupFix~31bf3856ad364e35~amd64~~19041.5247.1.14\amd64_microsoft-windows-edgechromium_31bf3856ad364e35_10.0.19041.3636_none_7437336f112dbb3d\edge.wim
0.11 GB - F:/Window